# Initalize libraries

## Import libraries

In [ ]:
import sys, os
import time
from os.path import join
from os import path
from importlib import reload
from getpass import getuser
from glob import glob
from tqdm.auto import tqdm

# Data
import xarray as xr
import h5py
import numpy as np
from nexusformat.nexus import *

# Plotting
import imageio
import matplotlib.pyplot as plt
import fabio
import skimage.morphology

# skimage
from skimage.draw import ellipse

# scipy
from scipy.ndimage.filters import gaussian_filter
import scipy
from scipy import stats
from scipy import ndimage

# pyFAI
import pyFAI
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
from pyFAI.detectors import Detector

# Self-written libraries
sys.path.append(join(os.getcwd(), "library"))
import reconstruct as reco
import fthcore as fth
import support_functions as sup
import interactive
from interactive import cimshow
import reconstruct_rb as rec
import reconstruct as reco

plt.rcParams["figure.constrained_layout.use"] = True  # replaces plt.tight_layout


In [ ]:
# Is there a GPU?
try:
    # Cupy
    import cupy as cp
    import cupyx as cpx

    GPU = True

    print("GPU available")

    # Self-written library
    import CCI_core_cupy as cci
    import Phase_Retrieval as PhR
except:
    GPU = False
    import CCI_core as cci

    print("GPU unavailable")

In [ ]:
# interactive plotting
import ipywidgets

%matplotlib widget

# Auto formatting of cells
#%load_ext jupyter_black

## Experiment specific Functions

In [ ]:
PROPOSAL = 11018955
USER = getuser()

### Loading data

In [ ]:
BASEFOLDER = "/asap3/petra3/gpfs/p04/2024/data/%s/" % PROPOSAL
sample_name = "2403_tomo"


# Load any kind of data from collection
def load_pre_scan_snapshot(scan_id, key):
    fname = join(BASEFOLDER, "raw", "%s_%05d.h5" % (sample_name,scan_id))
    entry = "entry%d"%scan_id
    
    with h5py.File(fname, "r") as f:
        data = np.array(f[entry]["measurement"]["pre_scan_snapshot"][key][()])
        print("Loaded: %s" % (fname))
    return np.squeeze(data)

# Load any kind of data from measurements
def load_data(scan_id, key):
    fname = join(BASEFOLDER, "raw", "%s_%05d.h5" % (sample_name,scan_id))
    entry = "entry%d"%scan_id

    with h5py.File(fname, "r") as f:
        data = np.array(f[entry]["measurement"][key][()])
        print("Loaded key %s of %s" % (key, entry))
    return np.squeeze(data)


def get_image_id(im_id):
    fname = join(BASEFOLDER, "raw", "%s_%05d.h5" % (sample_name,im_id))
    entry = "entry%d"%im_id

    with h5py.File(fname, "r") as f:
        spe_id = str(f[entry]["measurement"]["ccd2"][()])[-10:-6]
    return spe_id

def load_spe_files(spe_id):
    fname = join(BASEFOLDER, "raw", sample_name +"  " +str(spe_id) + ".spe")
    im_out = np.array(imageio.mimread(fname, memtest="5000MB"))

    return im_out
    
# Load image files
def load_images(im_id):
    """
    Load ccd images from nxs files
    """
    
    spe_id = get_image_id(im_id)
    im_out = load_spe_files(spe_id)
    
    return im_out


# Full image loading procedure
def load_processing(im_id, crop=None):
    """
    Loads images, averaging of two individual images (scans in tango consist of two images),
    padding to square shape, Additional cropping (optional)
    """

    # Load data
    images = load_images(im_id)

    # Zeropad to get square shape
    images = sup.padding(images)

    # Calculate mean
    if images.ndim > 2:
        image = np.mean(images, axis=0)
    else:
        image = images.copy()

    # Optional cropping
    if crop is not None:
        images = images[:, :crop, :crop]
        image = image[:crop, :crop]

    return image

### Loading, saving fth & cdi data

In [ ]:
# Saving of log files for fth and cdi recos
def save_fth_h5():
    # Save h5
    data = {}
    data["im_id"] = im_id
    data["dark_id"] = dark_id
    data["im_centered"] = im_c
    data["recon"] = recon
    data["center"] = center
    data["roi"] = roi
    data["prop_dist"] = prop_dist
    data["phase"] = phase
    data["mask_bs"] = mask_pixel
    data["experimental_setup"] = experimental_setup

    filename = join(
        folder_general, "Logs", "Data_ImId_%04d_fth_%s" % (im_id, USER)
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)


def save_cdi_h5():
    # Save h5
    data = {}
    data["im_id"] = im_id
    data["dark_id"] = dark_id
    data["pos"] = pos
    data["center"] = center
    data["roi"] = roi
    data["prop_dist"] = prop_dist_cdi
    data["phase"] = phase_cdi
    data["mask_bs"] = mask_bs_cdi
    data["supportmask"] = supportmask
    data["mask_pixel"] = mask_pixel
    data["reco"] = p
    data["reco_pc"] = p_pc
    data["experimental_setup"] = experimental_setup

    filename = join(
        folder_general,
        "Logs",
        "Data_ImId_%04d_cdi_%s" % (im_id, USER),
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)
    return

def load_cdi(im_id):
    """
    Load cdi dataset
    """
    fname = join(
        folder_general,
        "Logs",
        "Data_ImId_%04d_cdi_%s.hdf5" % (im_id, USER),
    )

    print("Loading: %s"%fname)

    
    with h5py.File(fname, "r") as f:
        data = {}
        data["im_id"] = f["im_id"][()]
        data["roi"] = f["roi"][()]
        data["prop_dist"] = f["prop_dist"][()]
        data["phase"] = f["phase"][()]
        data["supportmask"] = f["supportmask"][()]
        data["reco"] = f["reco"][()]
        data["reco_pc"] = f["reco_pc"][()]
        #data["srotx"] = f["experimental_setup"]["srotx"][()]
        data["srotz"] = f["experimental_setup"]["srotz"][()]
    return data

def load_fth(im_id):
    """
    Load fth dataset
    """
    fname = join(
        folder_general,
        "Logs",
        "Data_ImId_%04d_fth_%s.hdf5" % (im_id, USER),
    )
    
    with h5py.File(fname, "r") as f:
        data = {}
        data["im_id"] = f["im_id"][()]
        data["dark_id"] = f["dark_id"][()]
        data["im_centered"] = f["im_centered"][()]
        data["center"] = f["center"][()]
        data["roi"] = f["roi"][()]
        data["prop_dist"] = f["prop_dist"][()]
        data["phase"] = f["phase"][()]
        data["mask_bs"] = f["mask_bs"][()]
        data["recon"] = f["recon"][()]
        data["experimental_setup"] = f["experimental_setup"][()]
    return data

### Masking

In [ ]:
from matplotlib.path import Path


def create_single_polygon_mask(shape, coordinates):
    """
    Creates a polygon mask from coordinates of corner points

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    coordinates: nested list
        coordinates of polygon corner points [(yc_1,xc_1),(yc_2,xc_2),...]


    Output
    ======
    mask: array
        binary mask where filled polygon is "1"
    ======
    author: ck 2023
    """

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    x, y = x.flatten(), y.flatten()

    points = np.vstack((x, y)).T

    path = Path(coordinates)
    mask = path.contains_points(points)
    mask = mask.reshape(shape)
    return mask


def create_polygon_mask(shape, coordinates):
    """
    Creates multiple polygon masks from set of coordinates of corner points

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    coordinates: nested list
        coordinates of polygon corner points for multiple polygons
        [[(yc_1,xc_1),(yc_2,xc_2),...],[(yc_1,xc_1),(yc_2,xc_2),...]]

    Output
    ======
    mask: array
        binary mask where filled polygons are "1"
    ======
    author: ck 2023
    """

    if len(coordinates) == 1:
        mask = create_single_polygon_mask(shape, coordinates[0])

    # Loop over coordinates
    elif len(coordinates) > 1:
        mask = np.zeros(shape)
        for coord in coordinates:
            mask = mask + create_single_polygon_mask(shape, coord)
            mask[mask > 1] = 1

    return mask


def load_poly_masks(shape, polygon_name_list):
    """
    Loads set of polygon masks based on stored coordinates

    Parameter
    =========
    shape : tuple
        shape of output mask
    polygon_name_list : list
        keys of different mask coordinates to load

    Output
    ======
    mask: array
        binary mask where filled polygons are "1"
    ======
    author: ck 2023
    """

    mask = []

    # Load dictionary of coordinates
    mask_coordinates = load_poly_coordinates()

    # Loop over relevant mask keys
    for polygon_name in polygon_name_list:
        coord = mask_coordinates[polygon_name]
        mask.append(create_polygon_mask(im_c.shape, coord).astype(float))

    # Combine all individual mask layers
    mask = np.array(mask)
    mask = np.sum(mask, axis=0)
    mask[mask > 1] = 1

    return mask

In [ ]:
def auto_shift_mask(
    mask,
    image,
    shift_range_y=(-10, 10),
    shift_range_x=(-10, 10),
    step_size=1,
    crop=None,
    method="minimize",
):
    """
    Automatically shifts a binary (beamstop) mask to the optimal position

    Parameter
    =========
    mask : array
        binary mask, e.g., beamstop mask
    image : array
        image partially covered by mask
    shift_range_y, _x : tupel
        Min and max limit for search area in y- and x-direction
    step_size : scalar
        step size of scan area
    crop : int
        additional cropping of arrays to speed up calculations
        Crops according to [crop:-crop,crop:-crop]
    method : string
        method used for optimization. 'minimize' or 'maximize' metric <mask,image>

    Output
    ======
    optimized shift: tupel
        shift vector for optimized posistion
    mask_shifted: array
        mask shifted according to best shift vector
    overlap : array
        computed evaluation metric
    ======
    author: ck 2023
    """

    # Basic looping
    # Create set of arrays for shifting
    yshift = np.arange(shift_range_y[0], shift_range_y[1], step_size)
    xshift = np.arange(shift_range_x[0], shift_range_x[1], step_size)

    # Setup loss function to evaluate overlap
    overlap = np.zeros((yshift.shape[0], xshift.shape[0]))

    # Optinal cropping to reduce computation time
    if crop is not None:
        tmask, timage = mask[crop:-crop, crop:-crop], image[crop:-crop, crop:-crop]
    else:
        tmask, timage = mask.copy(), timage.copy()

    # Loop over all combinations of shifts
    for i, y in enumerate(yshift):
        for j, x in enumerate(xshift):
            # Shift mask to new position
            mask_shift = cci.shift_image(tmask, [y, x])

            # Calculate overlap
            overlap[i, j] = np.sum(mask_shift * timage)

    # Get best shift
    if method == "minimize":
        idx = np.unravel_index(
            np.argmin(
                overlap,
            ),
            overlap.shape,
        )
    elif method == "maximize":
        idx = np.unravel_index(
            np.argmax(
                overlap,
            ),
            overlap.shape,
        )

    # Output
    optimized_shift = (yshift[idx[0]], yshift[idx[1]])
    print(
        "Best mask overlap for shift: [%.2f,%.2f]"
        % (optimized_shift[0], optimized_shift[1])
    )

    # Shift
    mask_shifted = cci.shift_image(mask, optimized_shift)

    # Binarize mask (necessary for sub-px shift)
    mask_shifted[mask_shifted > 0.5] = 1
    mask_shifted[mask_shifted < 0.5] = 0

    return optimized_shift, mask_shifted, overlap

In [ ]:
def create_supportmask(support_coordinates, shape):
    """
    Create cdi support mask from a combination of multiple circular apertures

    Parameter
    =========
    support_coordinates: nested list
        Contains center coordinates and radius of each aperture [[yc_1,xc_1,r_1],[yc_2,xc_2,r_2],...]
    shape : int tuple
        shape/dimension of output array

    Output
    ======
    supportmask: array
        composed binary mask where circular apertures are "1"
    ======
    author: ck 2023

    """

    # Create support mask
    supportmask = np.zeros(shape)
    for i in range(len(support_coordinates)):
        supportmask += cci.circle_mask(
            supportmask.shape,
            [support_coordinates[i][0], support_coordinates[i][1]],
            support_coordinates[i][2],
        )

    return supportmask

In [ ]:
def create_ellipse_supportmask(support_coordinates, shape):
    """
    Create cdi support mask as a combination of multiple ellipses

    Parameter
    =========
    support_coordinates: nested list
        Contains center coordinates, height, width and rotation angle of each aperture 
        [[(yc_1,xc_1),height_1,width_1,angle_1],[(yc_2,xc_2),height_2,width_2,angle_2],...]
    shape : int tuple
        shape/dimension of output array

    Output
    ======
    supportmask: array
        composed binary mask where ellipses apertures are "1"
    ======
    author: ck/sg 2024

    """

    # Create support mask
    supportmask = np.zeros(shape)
    for i in range(len(support_coordinates)):
        center, height, width, angle = support_coordinates[i][0], support_coordinates[i][1], support_coordinates[i][2], support_coordinates[i][3]
        yy, xx = ellipse(center[1],center[0],height/2,width/2,rotation=-angle) 
        supportmask[yy,xx] = 1
    
    return supportmask

### Other

In [ ]:
def reconstruct_heraldo(holo,prop_dist,phase,experimental_setup,heraldo_axis=0):
    image = cci.reconstruct(holo)
    image = np.gradient(image,axis=heraldo_axis)
    image = cci.FFT(image)
    image = fth.propagate(image, prop_dist*1e-6, experimental_setup = experimental_setup)*np.exp(1j*phase)
    image = fth.reconstruct(image)
    
    return image

In [ ]:
# Worker which performs complete fth reconstruction process
def worker(image):
    # Centering
    shift_c = np.array(image.shape) / 2 - center
    im_c = cci.shift_image(image, shift_c)

    # Apply beamstop to image
    im_b = im_c * mask_bs

    # Create masks
    mask = load_poly_masks(im_c.shape, polygon_names)

    # Optimize position of drawn mask relative to target image
    if optimize_position is True:
        # level 1 (rough)
        optimized_shift, _, _ = auto_shift_mask(
            mask,
            im_c,
            shift_range_y=[-10, 10],
            shift_range_x=[-10, 10],
            step_size=2,
            crop=300,
        )
        # Level 2 (fine)
        optimized_shift, mask_shifted, overlap = auto_shift_mask(
            mask,
            im_c,
            shift_range_y=[optimized_shift[0] - 4, optimized_shift[0] + 4],
            shift_range_x=[optimized_shift[1] - 4, optimized_shift[1] + 4],
            step_size=0.5,
            crop=300,
        )

    # optional. More mask functions

    # BS center part
    mask_bs_center = cci.circle_mask(
        image.shape, np.array(image.shape) / 2, bs.rBS, sigma=None
    )

    # Create image specific beamstop mask
    mask_pixel = mask.copy()
    mask_pixel = mask_pixel + (im_c > 40000) + mask_bs_center
    mask_pixel[mask_pixel > 1] = 1

    # Create smooth mask
    footprint = skimage.morphology.disk(6)
    mask_pixel_smooth = skimage.morphology.dilation(mask_pixel, footprint)
    mask_pixel_smooth = gaussian_filter(mask_pixel_smooth, 4)

    # Do the reconstruction
    holo = im_b*(1-mask_pixel_smooth)
    recon = np.zeros(image.shape, dtype=np.complex_)

    # Reconstruct
    if heraldo is True:
        recon = reconstruct_heraldo(holo,prop_dist,phase,experimental_setup, heraldo_axis = 1)
    elif heraldo is False:
        recon = cci.reconstruct(
            fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
            * np.exp(1j * phase)
        )

    # worker dictionary
    worker_dict = {}
    worker_dict["im_c"] = im_c
    worker_dict["holo"] = holo
    worker_dict["recon"] = recon
    worker_dict["mask_pixel"] = mask_pixel
    
    return worker_dict

In [ ]:
# Setup phase and propagation for cdi once
phase_cdi = 0
prop_dist_cdi = 0
dx = 0
dy = 0


def phase_retrieval(
    pos, mask_pixel, supportmask, vmin = 0, Startimage=None, Startgamma=None
):
    # Prepare Input holograms
    pos2 = pos.copy()

    mi, _ = np.percentile(pos2[pos2 != 0], [vmin, 99.9])
    pos2 = pos2 - mi

    pos2[pos2 < 0] = 0
    pos2 = pos2.astype(complex)

    bsmask_p = mask_pixel.copy()
    bsmask_p[pos2 <= 0] = 1

    # Setup start image and startgamma
    if Startimage is None:
        Startimage = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(supportmask)))
    else:
        Startimage = Startimage.copy()
    if Startgamma is None:
        Startgamma = np.ones(pos.shape) * 1e-6 * 2
        Startgamma[pos.shape[0] // 2, pos.shape[1] // 2] = 0.7
    else:
        Startgamma = Startgamma.copy()

    # Settings for phase retrieval reconstructions
    partial_coherence = True

    # Setup
    retrieved_p = np.zeros(pos2.shape, np.cdouble)
    retrieved_n = np.zeros(pos2.shape, np.cdouble)

    # Algorithms and Inital guess
    plt.rcParams["figure.dpi"] = 100
    print("CDI - larger mask")

    algorithm_list = ["mine", "mine", "mine"]
    Nit_list = [700, 50, 50]  # iterations for algorithm_list

    x = (np.sqrt(np.maximum(pos2, np.zeros(pos2.shape)))[mask_pixel == 0]).flatten()
    y = ((np.abs(Startimage))[mask_pixel == 0]).flatten()
    res = stats.linregress(x, y)
    Startimage -= res.intercept
    Startimage /= res.slope

    average_img = 30
    real_object = False  # always set to False

    if partial_coherence:
        RL_freq = 20
        RL_it = 50

        algorithm_list_pc = ["mine", "ER", "ER"]
        Nit_list_pc = [700, 50, 50]

    # Execute Phase retrieval
    start_time = time.time()
    for i in range(len(Nit_list) // 3):
        print("############ -   CDI")

        # Positive helicity - beta_mode="arctan"
        retrieved_p, Error_diff_p, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(pos2, np.zeros(pos2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i],
            beta_zero=0.5,
            Nit=Nit_list[3 * i],
            beta_mode="arctan",
            plot_every=349,
            Phase=Startimage,
            seed=False,
            real_object=real_object,
            bsmask=bsmask_p,
            average_img=average_img,
            Fourier_last=True,
        )

        # Positive helicity - beta_mode="const"
        retrieved_p, Error_diff_p2, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(pos2, np.zeros(pos2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i + 1],
            beta_zero=0.5,
            Nit=Nit_list[3 * i + 1],
            beta_mode="const",
            plot_every=24,
            Phase=retrieved_p,
            seed=False,
            real_object=real_object,
            bsmask=bsmask_p,
            average_img=average_img,
            Fourier_last=True,
        )


        print("--- %s seconds ---" % np.round((time.time() - start_time), 2))

        Startimage = retrieved_p.copy()

        # Partial coherence phase retrieval
        if partial_coherence:
            # CDI_PC
            print("############   -   CDI_pc")
            pos3 = (np.abs(retrieved_p) ** 2) * bsmask_p + np.maximum(
                pos2, np.zeros(pos2.shape)
            ) * (1 - bsmask_p)

            # retrieve pos image
            (
                retrieved_p_pc,
                Error_diff_p_pc,
                Error_supp,
                gamma_p,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(pos3),
                mask=supportmask,
                mode=algorithm_list_pc[3 * i],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i],
                beta_mode="arctan",
                gamma=Startgamma,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=349,
                Phase=Startimage,
                seed=False,
                real_object=False,
                bsmask=np.zeros(bsmask_p.shape),
                average_img=average_img,
                Fourier_last=True,
            )

            (
                retrieved_p_pc,
                Error_diff_p_pc2,
                Error_supp,
                gamma_p,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(pos3),
                mask=supportmask,
                mode=algorithm_list[3 * i + 1],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i + 1],
                beta_mode="const",
                gamma=gamma_p,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=24,
                Phase=retrieved_p_pc,
                real_object=False,
                bsmask=np.zeros(bsmask_p.shape),
                average_img=average_img,
                Fourier_last=True,
            )

            print("--- %s seconds ---" % np.round((time.time() - start_time), 2))

            Startimage = retrieved_p_pc.copy()
            Startgamma = gamma_p.copy()

    print("Phase Retrieval Done!")

    return (
        retrieved_p,
        retrieved_p_pc,
        bsmask_p,
        gamma_p,
    )

In [ ]:
def optimize_phase_contrast(recon, supportmask, method="contrast", prefered_color=None):
    """
    Automatically shifts contrast of phase retrieval reconstruction into real part

    Parameter
    =========
    recon : complex array
        FTH/CDI reconstruction plane (Patterson map)
    supportmask : array
        Supportmask of Patterson map for phase retrieval
    method : string
        Choose method for phase optimization ("contrast","minima","maxima")
    prefered_color : string or None
        Shift contrast such that color of domains with largest are is white ("white"),
        black ("black") or non-specific (None)

    Output
    ======
    recon_optimized: complex array
        reconstruction with optimized contrast
    optimized_phase: complex scalar
        phase corresponding to optimized reconstruction
    ======
    author: ck 2023
    """

    # filter references from supportmask
    mask = supportmask.copy()
    mask = mask.astype(bool)
    mask = skimage.morphology.remove_small_objects(mask, min_size=200)
    mask = mask.astype(float)

    # Make object aperture smaller to minimize edge effects
    footprint = skimage.morphology.disk(4)
    mask = skimage.morphology.erosion(mask, footprint)

    # Gaussian filter to remove high intensity peaks
    reco = scipy.ndimage.gaussian_filter(recon, 1)

    # Different functions for optimization
    def contrast(phi, reco, tmask):
        temp = np.imag(reco * np.exp(1j * phi)) * tmask
        mi, ma = np.percentile(temp[temp != 0], [1, 99])
        contrast = ma - mi
        return contrast

    def minima(phi, reco, tmask):
        tmp = np.real(reco * np.exp(1j * phi)) * tmask
        minima, maxima = np.percentile(tmp[tmp != 0], [0.01, 99])
        return minima

    def maxima(phi, reco, tmask):
        tmp = np.real(reco * np.exp(1j * phi)) * tmask
        minima, maxima = np.percentile(tmp[tmp != 0], [0.01, 99])
        return maxima

    # Choose optimization method
    if method == "minima":
        optimized_phase = scipy.optimize.fminbound(
            minima, -np.pi, np.pi, args=(recon, mask), disp=False
        ).astype(float)
    elif method == "contrast":
        optimized_phase = scipy.optimize.fminbound(
            contrast, -np.pi, np.pi, args=(recon, mask), disp=False
        ).astype(float)
    elif method == "max":
        optimized_phase = scipy.optimize.fminbound(
            contrast, -np.pi, np.pi, args=(recon, mask), disp=False
        ).astype(float)

    # Calc optimized reconstruction
    recon_optimized = recon * np.exp(1j * optimized_phase)

    # Optional: Shift phase such that "background" are white or black
    mi_p, ma_p = np.percentile(recon_optimized[mask == 1], [1, 99])
    mean = np.mean(recon_optimized[mask == 1])
    if prefered_color == "white":
        # Make white domains the dominant domains
        if mean < (mi_p + ma_p) / 2:
            optimized_phase = optimized_phase + np.pi
            recon_optimized = recon * np.exp(1j * optimized_phase)
    elif prefered_color == "black":
        # Make black domains the dominant domains
        if mean > (mi_p + ma_p) / 2:
            optimized_phase = optimized_phase + np.pi
            recon_optimized = recon * np.exp(1j * optimized_phase)

    return recon_optimized, optimized_phase

In [ ]:
def correct_background(image):
    offset = np.quantile(image[80:120],0.1,axis=0)
    mask_vert = np.ones(image.shape)
    image = image - offset * mask_vert

    #offset = np.quantile(image[-40:],0.1,axis=0)
    #mask_vert = np.ones(image.shape)
    #mask_vert[int(image.shape[0]/2):] = 0
    #image = image - offset * mask_vert

    image[56:59,:]  = 0
    
    return image

In [ ]:
import tomopy
def get_mask_projections(mask, theta):
    '''
    mask: 2D array
    theta: 1D list of angles in rad

    return projections of mask
    '''

    thick_mask = np.dstack(( np.zeros(mask.shape),mask, np.zeros(mask.shape)))
    proj = tomopy.project(np.rot90(thick_mask, axes=(1,2)), theta, pad=True)
    proj[proj>0]=1
    proj = proj[:,:,2:-2]
    proj = np.squeeze(proj)
    
    return proj

# Experimental Details

In [ ]:
# Dict with most basic experimental parameter
experimental_setup = {
    "ccd_dist": 0.09,  # ccd to sample distance
    "px_size": 13.5e-6,  # pixel_size of camera
    "binning": 1,  # Camera binning
}

# Setup for azimuthal integrator
detector = Detector(
    experimental_setup["binning"] * experimental_setup["px_size"],
    experimental_setup["binning"] * experimental_setup["px_size"],
)

# General saving folder and log folder
folder_general = sup.create_folder(join(BASEFOLDER, "processed"))
sup.create_folder(join(folder_general, "Logs"))
sup.create_folder(join(folder_general, "Topos"))
print("Output Folder: %s" % folder_general)

# Load images

Start by loading the images: image of interest (im), any kind of dark image (dark)

In [ ]:
# Define scan ids for each image
im_id = 1024
dark_id = 1025

heraldo = True

# Load energy and add to experimental setup
experimental_setup["energy"] = load_pre_scan_snapshot(im_id,"energy")
experimental_setup["lambda"] = cci.photon_energy_wavelength(
    experimental_setup["energy"], input_unit="eV"
)

# Load angles
experimental_setup["srotx"] = load_pre_scan_snapshot(im_id,"srotx")
experimental_setup["srotz"] = load_pre_scan_snapshot(im_id,"srotz")

print(experimental_setup)
print("Image Id: %s" % im_id)



## Load image of interest

In [ ]:
# Load image
image = load_processing(im_id, crop=None)

# Plot
fig, ax = cimshow(image)
ax.set_title("Image")

## Load dark image

In [ ]:
# Load image
if dark_id is not None:
    dark = load_processing(dark_id, crop=None)
    image = image - dark
    image = correct_background(image)

    # Plot
    fig, ax = cimshow(dark)
    ax.set_title("Dark")

In [ ]:
fig, ax = cimshow(image)
ax.set_title("Background subtracted image")

# Center holograms

* Find center of the hologram to get a well-defined q-space. 
* Create smooth mask for beamstop or overexposed areas in direct beam

## Basic widget to find center

Try to **align** the circles to the **center of the scattering pattern**. Care! Position of beamstop might be misleading and not represent the actual center of the hologram. Circles are just a guide to eye and will not be used otherwise.

In [ ]:
# Find center position via widget
c0, c1 = [1009, 1099]  # initial values
ic = interactive.InteractiveCenter(gaussian_filter(image,3), c0=c0, c1=c1)

In [ ]:
# Get center positions
center = [ic.c0, ic.c1]
print(f"Center:", center)

## Azimuthal integrator widget for finetuning
More of an "expert widget" which works very well for alignment if you have an Airy Pattern as a scattering image. PyFai transforms images from carthesian detector coordinate system into polar coordinate system with angle `phi` and radial distance `q` as axis (azimuthal transformation). The center of the coordinate system will be defined in the azimuthal integrator class and must not necessarily represents the center coordinates of your image array. If the center is set correctly, all rings of the Airy pattern will be transformed into a straight line in the I(q,chi)-plot as rings appear at a given q for all angles chi.

## Centering of image hologram

In [ ]:
# Apply to topo and image
shift_c = np.array(image.shape) / 2 - center
im_c = cci.shift_image(image, shift_c)

## Calculate Projection

In [ ]:
%%time
proj_im = PhR.inv_gnomonic(im_c, center=None, experimental_setup = experimental_setup, method='cubic' , mask=None)

In [ ]:
im_c = proj_im.copy()

# Create beamstops

We want to cover the beamstop with a smooth circle to cover its sharp edges as these would create ringing-like artifacts in the reconstruction plane. Make it only as large as necessary to keep as much information as possible.

## Draw Circle beamstop

Set beamstop diameter and std for smoothing filter. Higher values mean stronger smoothing. If you have a very small beamstop you might need to reduce the smoothing value. Otherwise the sharp gradient of the real beamstop will still be visible

In [ ]:
bs = interactive.InteractiveBeamstop(
    im_c, im_c.shape[0] / 2, im_c.shape[1] / 2, rBS=50, stdBS=4
)

In [ ]:
# Take value from widget and create beamstop mask
bs_diam = bs.rBS
bs_smoothing = bs.stdBS
mask_bs = 1 - cci.circle_mask(
    image.shape, np.array(image.shape) / 2, bs.rBS, sigma=bs_smoothing
)

# Apply beamstop to image
im_b = im_c * mask_bs

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(im_b, [0.1, 99.9])
ax[0].imshow(im_b, cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Masked image")

ax[1].imshow(mask_bs)
ax[1].set_title("Beamstop mask")

ax[2].imshow(1 - mask_bs)
ax[2].set_title("1 - Beamstop mask")


## Manual masking of beamstop wires

Just mask the beamstop wires, broken pixels, etc. 

In [ ]:
poly_mask = interactive.draw_polygon_mask(im_c)

In [ ]:
# Take poly coordinates and mask from widget
p_coord = poly_mask.coordinates
mask_draw = poly_mask.full_mask.astype(int)

print("Copy these coordinates into the 'load_poly_coordinates()' function:")
print(p_coord)

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(im_c * (1 - mask_draw), [0.1, 99.9])
ax[0].imshow(im_c * (1 - mask_draw), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Image * (1-mask_draw)")

mi, ma = np.percentile(im_c * mask_draw, [0.1, 99.9])
ax[1].imshow(im_c * mask_draw, vmin=mi, vmax=ma)
ax[1].set_title("Image * mask_draw")

ax[2].imshow(1 - mask_draw)
ax[2].set_title("1 - mask_draw")


In [ ]:
def load_poly_coordinates():
    """
    Dictionary that stores polygon corner coordinates of all drawn masks
    Example: How to add masks with name "test":
    mask_coordinates["test"] = copy coordinates from above
    """

    # Setup dictonary
    mask_coordinates = dict()

    # Setup dictonary
    mask_coordinates = dict()

    # Mask #1
    mask_coordinates["bs_medium"] = [[(1017.394483698999, -35.46145449788588), (1035.2276956836204, 505.3876493026536), (1047.325406869094, 994.0097061491854), (1059.1093028377018, 1016.5955067556837), (1459.190721041451, 991.086784173353), (2068.7721840663694, 944.2269827091939), (2067.1563286309156, 987.8550794664445), (1460.4026126180413, 1031.8871339185596), (1053.5446852969703, 1049.0012206693552), (1101.2358933146809, 2097.0416552096503), (1041.885818564897, 2104.3587877130485), (1023.8641950107121, 1398.6985283148524), (1012.3010494068428, 1055.2204990972316), (529.7787049731799, 1059.421969824299), (-49.71919730977076, 1055.9804064833297), (-45.08612508776184, 1014.2827564852495), (532.8674197878524, 1015.4077837152142), (1010.9917276325531, 1011.3582196585246), (992.2042825905437, 510.2399139372111), (963.1751999921164, -33.513456280872134)]]
    mask_coordinates["bs_medium_2"] = [[(972.9776892552021, 1004.2743085178313), (683.0854056548453, 1011.0191797381575), (429.3488120641165, 1014.0243786386102), (-5.38786287953252, 1010.0262182986162), (-11.784588413299048, 1027.546033470307), (-6.456709339024059, 1050.189519535976), (407.3336253818711, 1051.9815970562745), (785.5107592764842, 1048.7548363356034), (970.4099189292948, 1041.9313069708498), (977.0697677721386, 1049.9231255822624), (987.7255259206885, 1067.238732573656), (1005.041132912082, 1075.2305511850686), (1010.3690119863572, 1093.8781279450309), (1022.57787485063, 1452.2001151306752), (1023.891500713352, 1538.8489667863084), (1029.7269726228894, 1668.2234921518748), (1036.1520087654237, 1833.2762608301566), (1039.0507610028706, 2010.5397226740427), (1041.0931146480093, 2061.6873617870824), (1083.7161472422092, 2055.027512944239), (1082.3841774736404, 1997.7528128957829), (1061.329837800928, 1535.5648530012597), (1045.621811861143, 1081.8904000279126), (1059.0201832634914, 1068.6419757912079), (1080.0381970670428, 1045.6535231935734), (1250.4043869753773, 1036.9986877096228), (1711.4432772620323, 1003.3274407611157), (1930.818796336602, 981.3023122296238), (1968.2328715564279, 975.4216165269843), (2057.474846050534, 986.0773746755342), (2052.9860015697454, 945.8344154978255), (1975.482075669149, 939.2662861903159), (1901.4373645403866, 946.447411617374), (1619.8397671015532, 973.7708588493829), (1382.0734859488762, 992.1616209292713), (1192.6712743111323, 1001.972213913788), (1074.126880684794, 1006.2447473119142), (1066.1341462971013, 991.3164557652381), (1042.158690462864, 972.6688790052756), (1026.505884395897, 560.1120440507323), (1007.553206285446, -9.489556366445356), (970.9641323120604, -5.802018358814962), (976.3933883688477, 150.2045104582064), (980.8479680128388, 277.27966418395783), (990.3028627366012, 506.77436338214807), (997.1844288881059, 749.2817066564569), (1006.4751487546124, 969.463223155699), (984.8003220196999, 986.5403593710846)]]
    mask_coordinates["bs_large"] =[[(-9.52514267623522, 988.0244972208288), (133.8623449984891, 1003.0301645356255), (444.7643270650601, 1004.8352658735243), (677.4009610677929, 1006.3647572722471), (999.1891601517673, 998.0282754306933), (1008.9687632034429, 924.746257259271), (1002.856550157784, 753.6042842087398), (990.8526783102134, 392.799693733892), (977.5143073637275, -17.355212870551895), (1009.1929383616316, -5.684138292376701), (1027.5331984130498, 374.4594336824738), (1040.871569359536, 736.2627456059059), (1047.540754832779, 842.9697131777938), (1049.2080512010896, 983.0226081158966), (1113.8991502915467, 991.4702430486711), (1282.2960834909322, 986.4683539437389), (1529.0559460009226, 968.1280938923206), (1872.5189978729368, 933.1148701577949), (1955.8838162884742, 928.1129810528627), (2022.575671020904, 939.784055631038), (2029.2448564941471, 996.4721321536034), (1947.5473344469203, 974.7972793655637), (1627.4264317312568, 998.139428521914), (1333.9822709085652, 1019.8142813099539), (1133.9067067112755, 1031.485355888129), (955.5059953020256, 1034.8199486247506), (921.8266086621485, 1036.8207042667236), (678.7348313316545, 1045.0328414964195), (469.98929284993585, 1046.824482476588), (116.52246276805741, 1046.824482476588), (-6.8574684869379325, 1023.4823333202376)], [(-18.49404559764608, 785.86607528193), (-15.741149396111126, -20.73251176780832), (1721.3363537724376, -15.226719364738301), (2061.961377109028, -16.878457085658965), (2070.998269815253, 2083.925571530775), (-33.65294626451481, 2100.746691469454), (-41.88066798492048, 1173.7567109934555), (-50.10838970532615, 731.8366354770906), (43.139123125938, 737.3217832910489), (81.53515782116438, 62.64860221779031), (960.9090655017594, 46.10727357274732), (1962.9632828604786, 59.87175458042202), (2008.0241868014546, 946.3583113396406), (1981.6954772961565, 1969.1555403163811), (838.0421581597698, 2002.0664271980036), (56.04291820032506, 1965.3159367546796), (34.102326945909965, 1019.127939000208), (36.84490085271187, 742.1279744132175)]]
    mask_coordinates["bs_large_2"] =[[(-2.5086775209114265, 1033.1521507595412), (462.5106850256015, 1047.2212079066492), (716.4941903654963, 1055.3664515181326), (821.4295329342035, 1050.7075435593438), (1315.2912422480256, 1058.9385720479077), (1567.9431462107898, 1056.584405702532), (2063.325378790087, 1047.3157061119305), (2059.255245247832, 986.3969629829544), (1574.0729004151235, 989.2590341562225), (1071.5700027986707, 997.6082929549916), (1058.7059864142693, 849.2688655802834), (1065.5626473107302, 266.452689381122), (1066.5421702959388, -9.772792447724441), (1001.8936532721664, -7.81374647730695), (998.9550843165403, 420.23779805888387), (998.3722543978254, 831.5693599969302), (981.7208968914667, 973.1058988009786), (865.4419098489327, 984.3308651809253), (813.1192991686708, 984.3308651809253), (463.38395409533996, 981.5770435661748), (-1.027724137005336, 967.2497251757201)]]
    mask_coordinates["bs_tiny"] = [[(1031.1221816767775, 977.4340148145258), (1034.0037306716035, 977.2570775955452), (1037.7326765093844, 971.856949685674), (1031.4787790844405, 968.2530087967233), (1028.192832979809, 972.5989375157521), (1029.8583443983453, 977.0801403765646)], [(1030.2067999471637, 998.8865063528044), (1026.1788660124541, 1002.3844489803155), (1028.2988312412488, 1007.6843620523017), (1032.9627547445966, 1008.3203516209401), (1035.4007147577104, 1005.1404037777482), (1034.8707234505118, 999.2045011371235)]]
    mask_coordinates["bs_medium_3"] = [[(1001.3623554256386, -18.779394518638583), (998.6085338176418, 509.9615405411348), (998.6085338176418, 912.0194953360979), (979.3317825616633, 923.0347817680855), (965.5626745216788, 934.0500682000732), (940.7782800497067, 961.5882842800423), (929.762993617719, 978.1112139280237), (-23.059282749212798, 961.5882842800423), (-17.551639533218975, 1044.2029325199496), (915.9938855777344, 1052.4643973439404), (973.8241393456696, 1110.2946511118757), (989.636646665352, 1169.1022379530568), (973.1767948281389, 2066.1641724451656), (1047.2461280955988, 2060.6775551661303), (1060.9626712932766, 1122.4660004449727), (1091.1390663281675, 1108.749457247295), (1124.0587700025942, 1056.6265930961194), (2065.2066426674105, 1073.1956453004602), (2062.4141515325086, 975.4584555788865), (1132.5146036101073, 989.4209112533972), (1057.1173429677503, 919.6086328808444), (1065.8658092893436, -36.25744650553679)]]
    mask_coordinates["bs_bar"] = [[(1005.9758120602492, -8.242722090094162), (1006.2678908488904, 404.85086350503616), (997.2307791924852, 991.0451812755406), (993.8673058083219, 1026.361651809255), (991.1765261868088, 1307.2116625466838), (998.239820678471, 2056.761678204615), (1048.69192144092, 2053.3982048204516), (1052.3917418266954, 1587.557154826576), (1052.358514403577, 1374.0500808455627), (1056.5116461834118, 1024.3103445930478), (1064.8365948396604, 701.1137930744791), (1066.518332975188, -5.72011705197167)]]
    mask_coordinates["bs_tiny_2"] = [[(1017.866806601309, 149.77970139629696), (1017.9864760868489, 153.21022664844236), (1024.2092893349268, 153.0506673343891), (1024.0896198493867, 149.97915053886354)], [(1031.6293165413158, 965.2245984409541), (1026.8137667514804, 968.289039216304), (1029.6593189000196, 973.5423662597608), (1032.94264830218, 973.7612548865716), (1035.5693118239085, 969.3834823503574)], [(1030.3159847804516, 993.2423426727241), (1027.689321258723, 995.2123403140205), (1026.5948781246698, 998.495669716181), (1027.908209885534, 1002.2167763719631), (1030.7537620340731, 1003.9678853864486), (1033.3804255558016, 1001.3412218647202), (1034.0370914362336, 996.5256720748847)], [(1028.3459871391553, 1061.0978169840423), (1025.9382122442375, 1061.0978169840423), (1025.2815463638055, 1071.6044710709562), (1025.0626577369949, 1083.2055682919233), (1027.689321258723, 1082.9866796651127), (1028.7837643927767, 1074.668911846306)], [(1021.7379140721885, 978.9420126138665), (1018.8591114173319, 979.1819128351044), (1019.1469916828174, 982.0127354457135), (1020.8742732757314, 983.308196640399), (1022.9853952226263, 982.9243562864181), (1022.8894351341311, 979.0859527466092)], [(1033.1091845588721, 977.9824117289141), (1027.92733978013, 977.9824117289143), (1028.1672400013683, 981.9647554014658), (1029.2228009748158, 983.2602165961514), (1032.9652444261292, 983.2122365519037)], [(1029.2228009748155, 972.0808662864582), (1029.0308807978251, 980.2374738085518), (1029.0788608420728, 987.2425602687031), (1030.4702821252536, 988.8738817731218), (1032.1975637181674, 988.7299416403789), (1031.9576634969294, 983.5001168173894), (1032.9172643818818, 972.6086467731818)]]
    return mask_coordinates

In [ ]:
# Which drawn masks do you want to load? You can combine multiple masks from
# load_poly_coordinates(). Just add names of mask as strings to list like
# ["bs_small","bs_medium"]
polygon_names = ["bs_tiny_2"]
mask_draw = load_poly_masks(
    im_c.shape,
    polygon_names,
)

#footprint = skimage.morphology.disk(20)
#mask_draw = skimage.morphology.dilation(mask_draw, footprint) # increase size
#mask_draw = skimage.morphology.erosion(mask_draw, footprint) # decrease size

# The relative position of the drawn beamstop with respect to the actual beamstop
# might change due to sample change, sample drift, realignment, etc. The drawn
# beamstop mask would therefore not cover the actual beamstop position. This function
# aligns the beamstop mask with respect to the actual position
optimize_position = False

# Optimize position of drawn mask relative to target image
if optimize_position is True:
    # level 1 (rough)
    optimized_shift, _, _ = auto_shift_mask(
        mask_draw,
        im_c,
        shift_range_y=[-10, 10],
        shift_range_x=[-10, 10],
        step_size=2,
        crop=300,
    )
    # Level 2 (fine)
    optimized_shift, mask_shifted, overlap = auto_shift_mask(
        mask_draw,
        im_c,
        shift_range_y=[optimized_shift[0] - 4, optimized_shift[0] + 4],
        shift_range_x=[optimized_shift[1] - 4, optimized_shift[1] + 4],
        step_size=0.5,
        crop=300,
    )

    mask_draw = mask_shifted.copy()

# Shift mask
#mask_draw = np.round(cci.shift_image(mask_draw,[2,0]))

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(im_c * (1 - mask_draw), [0.1, 99.9])
ax[0].imshow(im_c * (1 - mask_draw), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Image * (1-mask_draw)")

mi, ma = np.percentile(im_c * mask_draw, [0.1, 99.9])
ax[1].imshow(im_c * mask_draw, vmin=mi, vmax=ma)
ax[1].set_title("Image * mask_draw")

ax[2].imshow(1 - mask_draw)
ax[2].set_title("1 - mask_draw")

## Finetuning of mask position

In [ ]:
# Use widget to shift and expand or shrink the mask
ss_mask = interactive.Shift_Scale_Mask(im_c,mask_draw)

In [ ]:
# Take mask, shift and scaling from widget
mask_draw, mask_shift, mask_scale = ss_mask.get_mask()

## Overview beamstops
Verify good beamstop alignment

In [ ]:
# Mask over-saturated pixel
mask_im = (im_c > 40000)

mask_pixel = mask_draw + mask_im
mask_pixel = mask_pixel + cci.circle_mask(image.shape,[1026,1026],24)
mask_pixel[mask_pixel>1] = 1 

# Create smooth mask
footprint = skimage.morphology.disk(6)
mask_pixel_smooth = skimage.morphology.dilation(mask_pixel, footprint)
mask_pixel_smooth = gaussian_filter(mask_pixel_smooth, 4)

# Plot both
fig, ax = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
mi, ma = np.percentile(im_c, [1, 99.9])
ax[0, 0].imshow(im_c*(1-mask_pixel), vmin=mi, vmax=ma)
ax[0, 0].set_title("Image")
mi, ma = np.percentile(im_c * mask_pixel, [1, 99.99])
ax[0, 1].imshow(im_c * mask_pixel, vmin=mi, vmax=ma)
ax[0, 1].set_title("Image*mask")
ax[1, 0].imshow(mask_pixel)
ax[1, 0].set_title("Mask_pixel")
ax[1, 1].imshow(mask_pixel_smooth)
ax[1, 1].set_title("mask_pixel_smooth")

In [ ]:
# Plot an example of hologram
fig, ax = cimshow(im_c * (1 - mask_pixel_smooth))
ax.set_title(f" Im Id %d" % im_id)

# Reconstruct Holo (FTH)

Reconstruct the hologramm.
1. Choose a region of interest (ROI) which means selecting one reconstruction from the reconstruction plane.
2. Propagate the image and shift the phase for maximal contrast and sharpness in your ROI

## Set Patterson Map ROI

Choose the reconstructions as the ROI.

1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
xticks, yticks = [1090,1100], []
shift_rotate = interactive.Shift_Rotate(im_c, shift = (xticks, yticks), angle = 10, ticks = (yticks,xticks))

In [ ]:
if heraldo is True:
    _, _, heraldo_rotation = shift_rotate.get_parameter()
    tmp = fth.reconstruct(im_c * (1 - mask_pixel_smooth))
    tmp = ndimage.rotate(tmp,heraldo_rotation,reshape=False)
    tmp = np.gradient(tmp,axis=1)
    tmp = np.real(tmp)
else:
tmp = np.real(fth.reconstruct(im_c * mask_bs*(1 - mask_pixel_smooth)))
    
fig, ax = cimshow(tmp, cmap="gray")

In [ ]:
# Execute to get roi
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()

roi = np.array([y1, y2, x1, x2]).astype(int)  # ystart, ystop, xstart, xstop
#roi = [ 478,  801,  930, 1365]
roi_s = np.s_[roi[0] : roi[1], roi[2] : roi[3]]
print(f"Roi Reco:{roi}")

In [ ]:
# Correct rotation angle to align heraldo object
if heraldo is True:
    im_c = ndimage.rotate(im_c,heraldo_rotation,reshape = False)
    mask_pixel = ndimage.rotate(mask_pixel,heraldo_rotation,reshape = False)
    mask_pixel_smooth = ndimage.rotate(mask_pixel_smooth,heraldo_rotation,reshape = False)

## Tune propagation and phase
Focus the image by tuning the propagation distance. This really works like focussing in a microscope.
Phase slider will move contrast between real and imaginary part. Usually we use the phase which maximizes the contrast in the real part.

In [ ]:
# Widget
slider_prop, slider_phase, button = reco.propagate(
    im_c * mask_bs*(1 - mask_pixel_smooth),
    roi_s,
    phase = 0, #Initial value
    prop_dist = 0, #Initial value
    experimental_setup=experimental_setup,
    scale=(1, 99),
    heraldo = heraldo,
    heraldo_axis = 1
)

In [ ]:
# Read prop dist and phase from widget
prop_dist = slider_prop.value
phase = slider_phase.value

print(f"Propagation distance: %0.2f" % prop_dist)
print(f"Phase: %0.2f" % phase)

## Save reconstruction

Save png files of the images and a h5 file containing all important variables

In [ ]:
# Style of reconstruction plot
def plot_recon(recon, title):
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    fig.suptitle(title)

    vmin, vmax = np.percentile(np.real(recon), (1, 99))
    t_im1 = ax[0].imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[0].set_title("Real")
    #plt.colorbar(t_im1, ax=ax[0], aspect=50)

    vmin, vmax = np.percentile(np.imag(recon), (1, 99))
    t_im2 = ax[1].imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[1].set_title("Imag")
    #plt.colorbar(t_im2, ax=ax[1], aspect=50)

In [ ]:
def get_title(data_key,im_id,CDI=False):
    # Rotation in title
    #values = np.mean(np.array(load_pre_scan_snapshot(im_id, data_key)) * 1000)
    #values = [np.round(values, 2)]
    
    # Rotation in title
    values = load_pre_scan_snapshot(im_id, data_key)

    if CDI is False:
        title = "Image %d - FTH: %s = %.1f (%.1f deg)" % (im_id, data_key, values, values-67)
    elif CDI is True:
        title = "Image %d - CDI: %s = %.1f (%.1f deg)" % (im_id, data_key, values, values-67)
        
    return title

In [ ]:
# Create plot
holo = im_c * (1 - mask_pixel_smooth)
recon = np.zeros(image.shape, dtype=np.complex_)

# Reconstruct
if heraldo is True:
    recon = reconstruct_heraldo(holo,prop_dist,phase,experimental_setup, heraldo_axis = 1)
elif heraldo is False:
    recon = fth.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6*(1 - mask_pixel_smooth), experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )

# Create plot
title = get_title("srotz",im_id)
plot_recon(recon[roi_s], title)

# Save images
fname = join(
    folder_general,
    "Recon_ImId_%04d_%s.png" % (im_id, USER),
)
print("Saving: %s" % fname)
plt.savefig(fname, bbox_inches="tight", transparent=False)

# Save hdf5 file
save_fth_h5()

In [ ]:
# Closes all existing plots
plt.close("all")

# Batch processing FTH

# CDI Reconstruction

In [ ]:
# Copy values from FTH reco
pos = im_c.copy()

## Draw support mask

The support mask is the real-space constraint used for the (holographically-aided) phase retrieval, i.e., certain details about our sample like the sample geometry. For our samples we can directly derive a very strong constraint: The FTH reconstructions show us previsely the actual real-space sample structure, i.e., the arrangement of our aperture where x-rays are transmitted ("1") while the masked areas show no transmission ("0"). We will therefore create a binary mask that reflects this transmission as an input for the phase retrieval.

How to draw a support mask: Create a binary mask of the locations of sample apertures in the fth reconstruction. Areas with apertures are "1". Select only a single set of reconstructions (object & reference apertures) that originate from a single reference. Use the widget!

### Option 1: Execute if you want to create a new support mask

If you really want to create a new support mask, execute next cell and then the "InteractiveCircleCoordinates"-widget

In [ ]:
# How many references do you have?
nr_ref = 2

# Setup coordinates (nr_ref + 1 coordinates, as there is always the object aperture)
support_coordinates = [
    [(pos.shape[-2] // 2, pos.shape[-1] // 2), 10, 10, 0] for k in range(nr_ref + 1)
]

### Option 2: Execute if you want to load an existing support mask

In [ ]:
def get_supportmask_coordinates(sample):
    """
    Dictionary that stores coordinates of circular support mask apertures
    """

    # Setup dictonary
    support_coord = dict()

    # coordinates
    support_coord["s2305e-C5"] = [((945.5, 916.0), 72.5, 72.5, 0.0), ((941.5, 781.0), 10.0, 10.0, 0.0), ((818.5, 875.0), 10.0, 10.0, 0.0), ((867.5, 1022.5), 10.0, 10.0, 0.0), ((1073.5, 873.5), 10.0, 10.0, 0.0), ((1024, 1024), 10, 10, 0)]
    support_coord["s2402f_2-25deg"] = [((1110.0, 643.0), 223.5, 206.5, 0.0), ((1459.0, 1011.0), 22.5, 22.0, 0.0), ((1024, 1024), 10, 10, 0)]
    support_coord["s2402f_2-0deg"] = [((1101.5, 834.0), 120.5, 114.5, 141.0), ((1237.7, 1017.8), 12.0, 12.5, 0.0), ((1024, 1024), 10, 10, 0)]
    #support_coord["s2402f_2-35deg"] = [((1053.5, 839.0), 117.0, 100.0, 0.0), [(1187,1018),12,8,0], ((1024, 1024), 10, 10, 0)]
    support_coord["s2402f_2-35deg"] = [((984.0, 844.0), 110.0, 110.0, 0.0), ((725.3, 929.9), 10.0, 10.0, 0.0)]           
    return support_coord[sample]

In [ ]:
# Which supportmask to load? ("s2306a-C1", "s2308a-B1", ...)
sample = "s2402f_2-35deg"

# Get coordinates and create supportmask
support_coordinates = get_supportmask_coordinates(sample)

### Option 3: Adapt exisiting support mask with ellipse widget

In [ ]:
# Widget to find the positions and sizes of the different apertures
print("Cover the object & reference apertures for each set of reconstructions that originates from the same reference with circles.")
print("Optimization: Change one circle parameter, calc phase retrieval image, compare with images reconstructed with old circle parameter. Repeat!")

if heraldo is True:
    recon = np.real(reconstruct_heraldo(pos* mask_bs,prop_dist,phase,experimental_setup, heraldo_axis = 1))
elif heraldo is False:
    recon = np.real(fth.reconstruct(pos * mask_bs * np.exp(1j * phase) * (1 - mask_pixel_smooth)))

ds_ellipse = interactive.InteractiveEllipseCoordinates(recon,
    len(support_coordinates),
    coordinates=support_coordinates,
)

In [ ]:
# Take coordinates of circles from widget
support_coordinates = ds_ellipse.get_params()

# Create supportmask from coordinates
supportmask = create_ellipse_supportmask(support_coordinates, pos.shape)

# Add heraldo slit
supportmask[1018:1030,880:1024] = 1

# What to plot?
if heraldo is True:
    recon = np.real(reconstruct_heraldo(pos* mask_bs,prop_dist,phase,experimental_setup, heraldo_axis = 1))
elif heraldo is False:
    recon = np.real(fth.reconstruct(pos * mask_bs * np.exp(1j * phase) * (1 - mask_pixel_smooth)))

# Plot supportmask as overlay
fig, ax = plt.subplots(figsize=(6, 6))
mi, ma = np.percentile(recon, (1, 99))
ax.imshow(recon, vmin=mi, vmax=ma, cmap="gray")
ax.imshow(supportmask, alpha=0.4, cmap="binary")
ax.set_title("Image with overlayed mask")

### Load from cdi and calc projection

In [ ]:
tmp = load_cdi(997)["reco_pc"]
tmp = np.abs(tmp)*load_cdi(997)["supportmask"]

supportmask = tmp > 200000
footprint = skimage.morphology.disk(2)
supportmask= skimage.morphology.erosion(supportmask, footprint)
supportmask= skimage.morphology.dilation(supportmask, footprint)
supportmask = np.logical_not(skimage.morphology.remove_small_holes(np.logical_not(supportmask),area_threshold = 300))
supportmask = skimage.morphology.remove_small_holes(supportmask,area_threshold = 30)

#supportmask = gaussian_filter(supportmask.astype(float),1)
#supportmask = supportmask>0.5

fig, ax = plt.subplots(2,2,figsize=(12,12),sharex=True,sharey=True)
mi, ma = np.percentile(tmp[tmp!=0],[.1,99.9])
ax[0,0].imshow(tmp,vmin=mi,vmax=ma,cmap="gray")
ax[0,0].set_xlim([900,1300])
ax[0,0].set_ylim([750,1050])
ax[0,0].invert_yaxis()
ax[0,1].imshow(supportmask)

ax[1,0].imshow(tmp*supportmask,vmin=mi,vmax=ma,cmap="gray")
ax[1,1].imshow(tmp*(1-supportmask),vmin=mi,vmax=ma,cmap="gray")


In [ ]:
tmp = np.real(
        fth.reconstruct(pos * mask_bs * np.exp(1j * phase) * (1 - mask_pixel_smooth))
    )

# projection
mask = get_mask_projections(supportmask, [np.deg2rad(experimental_setup["srotz"]-67)])

mask = np.round(cci.shift_image(mask,[2,-34]))
footprint = skimage.morphology.disk(2)
#supportmask= skimage.morphology.erosion(supportmask, footprint)
supportmask= skimage.morphology.dilation(supportmask, footprint)
mask = mask + cci.circle_mask(mask.shape,np.array(mask.shape)/2,7)

fig, ax = plt.subplots(2,2,figsize=(12,12),sharex=True,sharey=True)
mi, ma = np.percentile(tmp[tmp!=0],[.1,99.9])
ax[0,0].imshow(tmp,vmin=mi,vmax=ma,cmap="gray")
ax[0,0].set_xlim([900,1300])
ax[0,0].set_ylim([750,1050])
ax[0,0].invert_yaxis()
ax[0,1].imshow(mask)

ax[1,0].imshow(tmp*mask,vmin=mi,vmax=ma,cmap="gray")
ax[1,1].imshow(tmp*(1-mask),vmin=mi,vmax=ma,cmap="gray")


In [ ]:
supportmask = mask.copy()

## Draw manually

In [ ]:
recon =  fth.reconstruct(
   fth.propagate(pos* mask_bs*(1 - mask_pixel_smooth), prop_dist * 1e-6, experimental_setup=experimental_setup)
    * np.exp(1j * phase))

In [ ]:
poly_mask = interactive.draw_polygon_mask(np.real(recon),cmap='gray')

In [ ]:
# Take poly coordinates and mask from widget
p_coord = poly_mask.coordinates

# Polygon mask
manual_supportmask = poly_mask.full_mask.astype(int)

# Heraldo reference
manual_supportmask[im_c.shape[0]/2-5:im_c.shape[0]/2+5,]

# Circular reference (optional)
#manual_supportmasl += cci.circle_mask(image.shape,(1024,1024), 8)


print("Copy these coordinates into the 'load_poly_coordinates()' function:")
print(p_coord)


In [ ]:
def load_poly_support():
    """
    Dictionary that stores polygon corner coordinates of all drawn masks
    Example: How to add masks with name "test":
    mask_coordinates["test"] = copy coordinates from above
    """

    # Setup dictonary
    mask_coordinates = dict()

    # Mask #1
    mask_coordinates["test1"] = [[(1050.5280838487358, 781.0857039912011), (1042.7753562601702, 782.5766131428483), (1027.2699010830388, 787.04934059779), (1022.4989917977675, 793.6093408650379), (1014.7462642092019, 798.6784319806386), (1010.5717185845895, 802.5547957749214), (1007.2917184509656, 806.1329777388748), (1008.1862639419539, 814.4820689880994), (1003.4153546566827, 825.5147967102891), (1003.7135364870121, 836.8457062628082), (1004.0117183173417, 845.7911611726918), (1003.7135364870121, 853.245706930928), (1008.7826276026128, 862.1911618408114), (1012.9571732272251, 870.5402530900361), (1019.517173494473, 878.8893443392608), (1029.0589920650154, 886.9402537581559), (1040.3899016175346, 891.4129812130976), (1055.298993134007, 892.6057085344154), (1068.1208118381735, 890.8166175524387), (1075.8735394267392, 883.9584354548614), (1085.117176166952, 877.100253357284), (1086.0117216579404, 866.9620711260827), (1086.9062671489287, 858.0166162161992), (1089.8880854522233, 847.5802521546684), (1089.8880854522233, 836.8457062628082), (1088.6953581309056, 824.9184330496303), (1091.9753582645294, 815.6747963094173), (1095.2553583981535, 806.7293413995337), (1093.1680855858472, 796.5911591683324), (1084.2226306759637, 793.9075226953673), (1074.382630275092, 788.8384315797667), (1063.3499025529022, 784.9620677854839), (1057.0880841159837, 780.7875221608716)]]
    mask_coordinates["test2"] = [[(1078.9983505042983, 792.582210113448), (1075.2163989407468, 792.9803102780323), (1068.0505959782283, 795.3689112655385), (1062.0790935094628, 799.5489629936743), (1058.0980918636192, 805.9185656270241), (1053.3208898886069, 810.6957676020364), (1053.1218398063147, 819.2549211406002), (1049.140838160471, 822.8378226218595), (1049.93703848964, 827.6150245968719), (1050.3351386542242, 833.7855771479294), (1050.136088571932, 838.1646789583574), (1049.5389383250556, 842.1456806042011), (1048.941788078179, 851.1029343073492), (1050.3351386542242, 853.2924852125632), (1055.5104407938209, 859.2639876813286), (1060.8847930157099, 871.6050927834438), (1064.8657946615535, 875.9841945938717), (1067.4534457313518, 878.5718456636702), (1073.4249482001171, 880.1642463220076), (1082.7803020678498, 883.5480977209746), (1088.3537043720307, 884.94144829702), (1092.7328061824587, 886.135748790773), (1101.2919597210225, 887.9271995314026), (1107.6615623543723, 887.5290993668183), (1114.0311649877221, 886.9319491199417), (1118.8083669627345, 884.3442980501434), (1121.794118197117, 885.1404983793121), (1128.5618209950512, 882.1547471449294), (1129.9551715710966, 879.5670960751311), (1136.9219244513229, 878.9699458282545), (1140.9029260971665, 876.3822947584562), (1142.6943768377962, 872.2022430303203), (1144.684877660718, 866.0316904792627), (1149.8601798003147, 856.6763366115302), (1153.0449811169897, 848.3162331552586), (1153.0449811169897, 843.3399810979541), (1151.4525804586522, 836.3732282177277), (1151.6516305409443, 826.619774185411), (1151.4525804586522, 818.8568209760159), (1149.063979471146, 812.4872183426661), (1152.6468809524051, 808.7052667791147), (1152.447830870113, 803.9280648041023), (1150.0592298826068, 801.5394638165961), (1147.8696789773928, 796.5632117592916), (1143.689627249257, 791.7860097842793), (1138.7133751919525, 790.9898094551105), (1136.7228743690307, 793.9755606894932), (1133.538073052356, 789.9945590436496), (1127.7656206658826, 788.8002585498966), (1125.7751198429607, 789.7955089613574), (1120.9979178679484, 788.4021583853122), (1116.2207158929361, 787.4069079738513), (1110.0501633418785, 785.6154572332216), (1098.7043086512242, 785.0183069863451), (1095.320457252257, 786.6107076446826), (1089.9461050303682, 786.4116575623904), (1084.9698529730638, 788.8002585498966)]]
    mask_coordinates["test3"] = [[(979.9684947052438, 788.886137686106), (975.099292303558, 789.0600377718805), (972.1429908453915, 789.929538200753), (969.0127893014507, 792.0163392300469), (966.0564878432842, 796.0160412028603), (963.2740864708924, 798.2767423179287), (957.883183811883, 800.0157431756736), (954.4051820963931, 802.4503443765166), (951.2749805524522, 806.2761462635555), (949.5359796947073, 809.4063478074963), (947.4491786654133, 812.5365493514372), (945.7101778076684, 815.1450506380546), (944.3189771214725, 819.1447526108681), (941.7104758348551, 821.2315536401619), (938.9280744624632, 823.4922547552304), (938.5802742909142, 828.0136569853672), (938.5802742909142, 832.1872590439551), (937.3629736904927, 836.013060930994), (937.0151735189437, 838.0998619602879), (937.3629736904927, 842.4473641046502), (938.5802742909142, 847.1426664205616), (938.9280744624632, 851.6640686506985), (937.8846739478162, 856.3593709666097), (939.9714749771101, 863.489274483364), (942.9277764352765, 868.8801771423733), (945.7101778076684, 873.7493795440591), (948.6664792658348, 877.227381259549), (950.0576799520308, 882.9660840901074), (954.7529822679421, 884.3572847763033), (961.7089856989219, 889.4003872637636), (968.4910890441272, 894.9651900085474), (974.5775920462345, 896.3563906947434), (980.8379951341162, 896.7041908662924), (984.8376971069297, 894.9651900085474), (986.9244981362235, 894.2695896654495), (994.4022018245269, 893.9217894939005), (997.8802035400167, 890.6176878641851), (1002.575505855928, 888.1830866633422), (1007.6186083433884, 888.5308868348911), (1011.0966100588782, 885.5745853767248), (1012.6617108308487, 880.35758280349), (1017.35701314676, 877.575181431098), (1020.6611147764754, 873.9232796298337), (1022.9218158915438, 867.6628765419518), (1023.2696160630928, 862.445873968717), (1027.4432181216807, 856.8810712239333), (1028.3127185505532, 844.5341651339442), (1028.660518722102, 837.2303615314154), (1026.3998176070338, 833.0567594728276), (1023.9652164061908, 827.1441565564947), (1021.7045152911223, 819.1447526108681), (1017.35701314676, 813.2321496945352), (1013.8790114312701, 808.5368473786239), (1010.0532095442313, 802.7981445480656), (1003.966706542124, 798.9723426610267), (999.0975041404381, 794.9726406882133), (996.8368030253697, 790.625138543851), (992.6632009667819, 789.0600377718805), (988.6634989939685, 789.7556381149785), (984.3159968496061, 789.4078379434295)]]
    mask_coordinates["test4"] = [[(1076.9721200665024, 785.0054531165506), (988.6435728309586, 784.2600645322843), (977.8354383590989, 786.1235359929499), (968.8907753479045, 790.2231732064139), (962.554972381642, 793.9501161277449), (954.3556979547138, 799.1678362176082), (946.5291178199188, 805.5036391838707), (941.3113977300554, 812.5848307343997), (936.4663719323253, 821.529493745594), (932.7394290109943, 835.6918768466516), (932.7394290109943, 843.1457626893135), (933.1121233031274, 853.5812028690402), (935.720983348059, 866.2528088015654), (940.9387034379224, 875.9428603970259), (948.3925892805843, 884.142134823954), (960.6915009209765, 894.9502692958138), (972.2450239771024, 900.5406836778102), (1079.9536744035672, 900.5406836778102), (1093.3706689203586, 897.1864350486123), (1103.4334148079522, 891.2233263744829), (1113.4961606955458, 880.0424976104899), (1118.341186493276, 869.9797517228964), (1120.9500465382077, 858.7989229589035), (1122.8135179988733, 847.6180941949107), (1122.068129414607, 835.6918768466516), (1119.4592693696754, 826.0018252511911), (1114.241549279812, 812.5848307343997), (1104.5514976843515, 801.4040019704067), (1097.0976118416895, 793.9501161277449), (1089.6437259990278, 788.3597017457485), (1083.680617324898, 786.1235359929499)]]
    mask_coordinates["drawn_mask_heraldo_-35deg"] = [[(983.4763791077349, 782.8878502755655), (973.1682003936147, 785.9421254501196), (962.4782372826752, 790.5235382119508), (958.6603933144825, 793.9595977833243), (954.0789805526513, 798.1592261483363), (948.7339989971815, 803.5042077038061), (943.3890174417118, 808.0856204656372), (933.4626231244107, 823.356996338408), (930.0265635530372, 833.6651750525282), (930.0265635530372, 842.4462161793715), (931.5537011403144, 851.6090417030339), (934.6079763148686, 861.1536516235155), (938.8076046798805, 870.6982615439973), (943.3890174417118, 879.4793026708405), (951.0247053780971, 887.1149906072259), (959.8057465049403, 892.8417565595149), (969.350356425422, 899.3320913054425), (976.9860443618074, 902.3863664799967), (1067.0871620111548, 902.768150876816), (1079.6860471061907, 900.8592288927196), (1089.994225820311, 897.8049537181654), (1097.6299137566964, 893.2235409563342), (1101.0659733280697, 888.2603437976837), (1105.2656016930819, 885.5878530199489), (1110.6105832485514, 874.51610551219), (1112.1377208358285, 868.4075551630817), (1117.100917994479, 856.5722388616844), (1119.773408772214, 845.5004913539256), (1119.0098399785754, 835.9558814334439), (1114.4284272167442, 824.1205651320465), (1109.4652300580938, 813.8123864179263), (1101.0659733280697, 800.0681481324326), (1089.2306570266724, 790.9053226087701), (1080.8314002966486, 786.7056942437582), (1070.141437185709, 782.5060658787462)]]
    return mask_coordinates

In [ ]:
# Which drawn masks do you want to load? You can combine multiple masks from
# load_poly_coordinates(). Just add names of mask as strings to list like
# ["bs_small","bs_medium"]
coord = load_poly_support()
polygon_names = "test4"
supportmask = create_polygon_mask(
    im_c.shape,
    coord[polygon_names],
)

# Slit
supportmask[1022:1026,910:1100] = 1

tmp = np.real(heraldo_reco.copy())

# projection
#mask = get_mask_projections(supportmask, [np.deg2rad(experimental_setup["srotz"]-67)])

# More preprocessing
#mask = np.round(cci.shift_image(mask,[1,-34]))
#footprint = skimage.morphology.disk(3)
#mask= skimage.morphology.erosion(mask, footprint)
#mask= skimage.morphology.dilation(mask, footprint)
#mask = mask + cci.circle_mask(mask.shape,np.array(mask.shape)/2,7)

fig, ax = plt.subplots(2,2,figsize=(12,12),sharex=True,sharey=True)
mi, ma = np.percentile(tmp[tmp!=0],[1,99])
ax[0,0].imshow(tmp,vmin=mi,vmax=ma,cmap="gray")
ax[0,0].set_xlim([900,1300])
ax[0,0].set_ylim([750,1050])
ax[0,0].invert_yaxis()
ax[0,1].imshow(supportmask)

ax[1,0].imshow(tmp*supportmask,vmin=mi,vmax=ma,cmap="gray")
ax[1,1].imshow(tmp*(1-supportmask),vmin=mi,vmax=ma,cmap="gray")


In [ ]:
fig, ax = cimshow(supportmask.astype(int))

### Take Roi
Choose the reconstructions as the ROI.

1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
roi_cdi = interactive.axis_to_roi(ax)
#roi_cdi = [450, 1099, 954, 1515]
#roi_cdi = [755,1070,972,1283]
##roi_cdi = [763,899,1032,1171]
#roi_cdi = [776,910,978,1121]
roi_cdi = [764,1039,702,1052]
#roi_cdi = np.s_[roi_cdi[0] : roi_cdi[1], roi_cdi[2] : roi_cdi[3]]
print("Sliced roi:", roi_cdi)

## Do Phase Retrieval

In [ ]:
# Executes the algorithm
#fth_reco = fth.reconstruct(pos * mask_bs * np.exp(1j * phase) * (1 - mask_pixel_smooth))
#startimage = cci.reconstruct(supportmask*np.abs(fth_reco))
(
    retrieved_p,
    retrieved_p_pc,
    bsmask_p,
    gamma_p,
) = phase_retrieval(pos, mask_pixel, supportmask, vmin = .5, Startimage=None, Startgamma=None)


## Reconstruct images from phase retrieval

In [ ]:
fig,ax=plt.subplots()
ax.imshow(np.abs(retrieved_p), vmax=10)
fig,ax=plt.subplots()
ax.imshow(np.abs(retrieved_p_pc), vmax=10)

In [ ]:
# New beamstop for CDI recos as phase retrieval of low-q might be insufficient. If phase retrieval worked well
# Try without beamstop: `use_bs = False`
use_bs = False
bs_diam_cdi = 32  # diameter of beamstop

# Create beamstop
if use_bs is True:
    mask_bs_cdi = 1 - cci.circle_mask(
        image.shape, np.array(image.shape) / 2, 10, sigma=4
    )
    mask_bs_cdi = 1 - mask_pixel_smooth.copy()
elif use_bs is False:
    mask_bs_cdi = np.ones(pos.shape)  # if you don't want a beamstop


# Get Reco
p = fth.reconstructCDI(
    fth.propagate(
        retrieved_p * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# Get Reco partial coherence
p_pc = fth.reconstructCDI(
    fth.propagate(
        retrieved_p_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# optimize phase
#_, phase_cdi = optimize_phase_contrast(p_pc, supportmask, method="contrast")

# Plotting
mode = "+"
print("Fine-tuning of reconstruction parameter:")
slider_prop, slider_phase, _, _= rec.focusCDI(
    retrieved_p_pc * mask_bs_cdi,
    np.zeros(retrieved_p_pc.shape),
    roi_cdi,
    mask=supportmask,
    phase=phase_cdi,
    dx=dx,
    dy=dy,
    prop_dist=prop_dist_cdi,
    experimental_setup=experimental_setup,
    operation=mode,
    max_prop_dist=2,
    scale=(1, 99.9),
)

In [ ]:
cimshow(np.abs(p_pc),cmap='gray')

In [ ]:
# Get phase from slider
phase_cdi = slider_phase.value
prop_dist_cdi = slider_prop.value

# Apply phase
#p = p * np.exp(1j * phase_cdi)
p_pc = p_pc * np.exp(1j * phase_cdi)

print("Phase CDI: %s" % phase_cdi)
print("Prop_dist: %s" % prop_dist_cdi)

In [ ]:
cimshow(np.abs(p_pc),cmap='gray')

In [ ]:
# Confirm that offset subtraction in cdi function works, i.e., only small fraction of hologram is actually masked
fig, ax = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)
tmp = np.abs(retrieved_p_pc)
mi, ma = np.percentile(tmp, [0.1, 99.9])
ax[0].imshow(tmp, vmin=mi, vmax=ma)
ax[0].set_title("Retrieved Holo")
ax[1].imshow(bsmask_p)
ax[1].set_title("Phase Retrieval mask")


## Save reconstructions

In [ ]:
# Style of reconstruction plot
def plot_recon(recon, title):
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    fig.suptitle(title)

    vmin, vmax = np.percentile(np.abs(recon)*supportmask[roi_cdi], (20, 99.9))
    t_im1 = ax[0].imshow(np.abs(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[0].set_title("Abs")
    #plt.colorbar(t_im1, ax=ax[0], aspect=50)

    vmin, vmax = np.percentile(np.real(recon)*supportmask[roi_cdi], (20, 99.9))
    t_im2 = ax[1].imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[1].set_title("Real")
    #plt.colorbar(t_im2, ax=ax[1], aspect=50)

In [ ]:
# Saves only real and imaginary part
recon = p_pc.copy()

# Plot
title = get_title("srotz", im_id, CDI=True)
plot_recon(recon[roi_cdi], title)

# Save images
fname = join(
    folder_general,
    "Recon_ImId_%04d_cdi_%s.png" % (im_id, USER),
)
print("Saving: %s" % fname)
plt.savefig(fname, bbox_inches="tight", transparent=False)

# Save h5
save_cdi_h5()

# Batch processing CDI

## Define Scan Ids

In [ ]:
# Load support mask of which sample?
sample = "s2305e-C5"

In [ ]:
# Define the sets for reconstructions. You can make a iterable list or use np.arange
im_id_set = [911]
dark_id_set = []

print("Image Set:  %s" % im_id_set)

## Execute Stack Reconstruction

In [ ]:
# Ugly Automatic processing of image stacks
for it, im_id in enumerate(tqdm(im_id_set)):
    # Load experimental setup
    experimental_setup["energy"] = 778#load_energy(im_id)
    experimental_setup["lambda"] = cci.photon_energy_wavelength(
        experimental_setup["energy"], input_unit="eV"
    )
    
    # Load angles
    #experimental_setup["s_rotx"] = load_data(im_id,"s_rotx")
    #experimental_setup["s_rotz"] = load_data(im_id,"s_rotz")
    experimental_setup["s_rotx"] = 0
    experimental_setup["s_rotz"] = 0
    
    # Load images
    image = load_processing(im_id)

    # Load image
    dark_id = dark_id_set[it]
    if dark_id is not None:
        dark = load_processing(dark_id, crop=None)
        image = image - dark

    # Process images
    worker_dict = worker(image)

    # Take values from worker
    recon = worker_dict["recon"]
    
    # Plotting
    title = get_title("m_magnett_read",im_id)
    plot_recon(worker_dict["recon"][roi_s], title)
        # Save images
    fname = join(
        folder_general,
        "Recon_ImId_%04d_fth_stack_%s.png" % (im_id, USER),
    )
    print("Saving: %s" % fname)
    plt.savefig(fname, bbox_inches="tight", transparent=False)

    # Save hdf5 file
    save_fth_h5()

    ################ CDI ###############
    # Create pos and neg helicity set
    pos = worker_dict["im_c"]

    # Get coordinates and create supportmask
    support_coordinates = get_supportmask_coordinates(sample)
    supportmask = create_ellipse_supportmask(support_coordinates, pos.shape)

    (
        retrieved_p,
        retrieved_p_pc,
        bsmask_p,
        gamma_p,
    ) = phase_retrieval(pos, mask_pixel, supportmask, vmin = 0, Startimage=None, Startgamma=None)

    # Get Reco
    p = fth.reconstructCDI(
        fth.propagate(
            retrieved_p * mask_bs_cdi,
            prop_dist_cdi * 1e-6,
            experimental_setup=experimental_setup,
        )
    )
    
    # Get Reco partial coherence
    p_pc = fth.reconstructCDI(
        fth.propagate(
            retrieved_p_pc * mask_bs_cdi,
            prop_dist_cdi * 1e-6,
            experimental_setup=experimental_setup,
        )
    )

    # Apply phase
    p = p * np.exp(1j * phase_cdi)
    p_pc = p_pc * np.exp(1j * phase_cdi)

    # Saves only real and imaginary part
    recon = p_pc.copy()
    
    # Plot
    title = get_title("m_magnett_read", im_id, CDI=True)
    plot_recon(recon[roi_cdi], title)
    
    # Save images
    fname = join(
        folder_general,
        "Recon_ImId_%04d_cdi_stack_%s.png" % (im_id, USER),
    )
    print("Saving: %s" % fname)
    print("")
    plt.savefig(fname, bbox_inches="tight", transparent=False)
    
    # Save h5
    save_cdi_h5()

In [ ]:
plt.close("all")